# LangChain Agent 이해하기 🤖

## 1. Agent란 무엇인가요? 
Agent는 주어진 질문이나 작업을 해결하기 위해 다양한 도구(Tools)를 사용할 수 있는 '지능형 비서'라고 생각하면 됩니다. 마치 우리가 계산이 필요할 때 계산기를 사용하고, 정보가 필요할 때 인터넷을 검색하는 것처럼, Agent도 필요한 도구를 선택하고 사용하여 문제를 해결합니다.

## 2. Agent의 주요 구성 요소 📦

### 2.1 도구 (Tools)
- **정의**: Agent가 사용할 수 있는 다양한 기능들
- **예시**: 
  - Calculator: 수학 계산을 수행
  - Wikipedia: 정보 검색을 수행
  - 그 외: 날씨 확인, 일정 관리, 이메일 전송 등

### 2.2 LLM (Large Language Model)
- **역할**: Agent의 '두뇌'
- **기능**: 
  - 질문 이해
  - 적절한 도구 선택
  - 결과 해석 및 답변 생성

### 2.3 프롬프트 템플릿
- **역할**: Agent의 '행동 지침서'
- **내용**:
  - 사용 가능한 도구 목록
  - 도구 사용 규칙
  - 답변 형식

## 3. Agent의 작동 프로세스 🔄

### 3.1 기본 실행 사이클
```
질문 → 도구 선택 → 도구 사용 → 결과 확인 → 최종 답변
```

### 3.2 상세 프로세스 예시
질문: "127*4 - 99는 얼마인가요?"

1. **질문 분석** 📝
   ```
   "이것은 수학 계산이 필요한 질문이군요!"
   ```

2. **도구 선택** 🛠
   ```
   Action: Calculator
   Action Input: 127*4 - 99
   ```

3. **도구 실행 및 관찰** 👀
   ```
   Observation: 409
   ```

4. **최종 답변 생성** ✍️
   ```
   Final Answer: 계산 결과는 409입니다.
   ```

## 4. 실제 코드에서의 구현 🖥️

### 4.1 코드 구조
```python
# 1. 도구 정의
tools = [Calculator, Wikipedia]

# 2. 프롬프트 템플릿 설정
prompt = "사용 가능한 도구들: ..."

# 3. Agent 설정
agent = LLMSingleActionAgent(...)

# 4. Agent 실행기 생성
agent_executor = AgentExecutor.from_agent_and_tools(...)
```

### 4.2 실행 제어
- **Stop 토큰** (`\nObservation:`):
  - Agent의 생각과 도구 실행을 구분
  - 실제 결과만 사용하도록 보장
  - 실행 흐름을 체계적으로 관리

## 5. Agent의 장점과 특징 🌟

### 5.1 장점
- 자동화된 의사결정
- 도구 조합을 통한 문제 해결
- 체계적인 실행 흐름
- 확장 가능한 구조

### 5.2 활용 분야
- 정보 검색 및 분석
- 자동화된 계산 및 데이터 처리
- 복합적인 작업 처리
- 사용자 질문 응답

### 1. 기본 설정 및 임포트

In [ ]:
# 필요한 라이브러리 설치
!pip install langchain openai wikipedia

주요 모듈 설명:

1. Tool과 AgentAction
   - Tool: 실제 작업을 수행하는 도구 정의
   - AgentAction: 특정 도구를 사용하는 행동을 표현

2. LLMChain과 PromptTemplate
   - PromptTemplate: 입력 형식 정의
   - LLMChain: 프롬프트와 LLM을 연결하여 응답 생성

3. AgentExecutor와 LLMSingleActionAgent
   - LLMSingleActionAgent: 행동 결정
   - AgentExecutor: 전체 실행 과정 관리

4. 환경 설정 (load_dotenv)
   - API 키와 같은 중요 정보를 안전하게 관리
   - .env 파일을 통한 설정 관리


In [9]:
from dotenv import load_dotenv  # .env 파일에서 환경 변수를 로드하는 함수
import os                      # 운영체제와 상호작용하기 위한 모듈

# .env 파일에서 환경 변수 로드
load_dotenv() 

# OpenAI API 키를 환경 변수에서 가져옴
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']   # 환경 변수에서 API 키 추출. 이 키는 OpenAI API 인증에 사용
# 사용할 GPT 모델 지정
GPT_MODEL = 'gpt-3.5-turbo'    # OpenAI의 GPT-3.5-turbo 모델 지정


In [12]:
# LangChain의 핵심 컴포넌트들을 임포트
from langchain.agents import (
    Tool,                    # 에이전트가 사용할 도구를 정의하는 클래스
    AgentExecutor,          # 에이전트의 실행을 관리하는 클래스
    LLMSingleActionAgent,   # 한 번에 하나의 행동을 수행하는 에이전트
    AgentOutputParser       # 에이전트 출력을 파싱하는 기본 클래스
)

from langchain.prompts import PromptTemplate    # 프롬프트 템플릿을 정의하는 클래스
from langchain.chains import LLMChain          # LLM과 프롬프트를 연결하는 체인 클래스
from langchain.llms import OpenAI             # OpenAI의 언어 모델을 사용하기 위한 클래스
from langchain.schema import (
    AgentAction,           # 에이전트가 수행할 행동을 정의하는 클래스
    AgentFinish            # 에이전트의 최종 응답을 나타내는 클래스
)
from langchain.utilities import WikipediaAPIWrapper  # Wikipedia API를 사용하기 위한 래퍼 클래스


### 2. 단일 도구(Tool) 테스트

Tool 클래스의 주요 특징:

1. 기본 구성 요소:
   - name: 도구의 식별자
   - func: 실제 실행될 함수
   - description: 도구의 설명

2. 실행 방식:
   - run() 메서드를 통해 직접 실행
   - Agent를 통한 자동 실행

3. 에러 처리:
   - 기본적으로 예외를 상위로 전파
   - handle_tool_error로 커스텀 에러 처리 가능

4. 반환 값:
   - func의 반환값을 그대로 전달
   - return_direct=True 설정 시 직접 반환

5. 사용 사례:
   - 정보 검색 (Wikipedia, 웹 검색 등)
   - 계산 (Calculator)
   - API 호출
   - 파일 처리
   - 데이터베이스 쿼리 등

주의사항:
1. API 제한 및 속도 제한 고려
2. 에러 처리 구현 필요
3. 적절한 설명 제공 중요


In [ ]:
### Tool(도구) 클래스 설명 및 예시

# Wikipedia API를 사용하기 위한 래퍼 클래스의 인스턴스 생성
wikipedia = WikipediaAPIWrapper(
    # 가능한 파라미터들:
    # lang="en"          # 검색 언어 설정 (기본값: "en")
    # top_k_results=3    # 반환할 최대 결과 수
    # doc_content_chars_max=2000  # 각 문서의 최대 문자 수
)

# Tool 클래스를 사용하여 Wikipedia 검색 도구 생성
wiki_tool = Tool(
    # 필수 파라미터
    name="Wikipedia",        # Agent가 도구를 선택할 때 사용하는 이름(도구의 고유 식별자)
    func=wikipedia.run,      # 도구가 실행할 실제 함수   
    
    # Agent가 이 설명을 보고 도구의 용도를 이해(도구에 대한 설명)
    description="Wikipedia에서 정보를 검색할 때 유용한 도구. 검색할 수 있는 주제나 키워드를 입력하세요.",

    # 선택적 파라미터
    # return_direct=False   # True면 도구의 출력을 직접 반환
    # handle_tool_error=None  # 도구 실행 중 오류 발생 시 처리할 함수
    # verbose=False         # 실행 과정의 상세 출력 여부

)
"""
return_direct 파라미터의 역할:

1. return_direct=False (기본값)
   - Agent가 도구의 결과를 받아서 처리
   - 결과를 바탕으로 추가 생각이나 행동 가능
   - Final Answer 형식으로 응답 생성
   - 더 자연스러운 대화형 응답 가능

2. return_direct=True
   - 도구의 결과를 즉시 최종 답변으로 반환
   - Agent의 추가 처리 없음
   - 결과가 그대로 출력
   - 빠른 응답이 필요한 경우 유용
"""


# 도구 사용 예시와 테스트
test_queries = [
    "BTS",         
    "인공지능 역사",  # 위키피디아의 검색에 적절하지 않은 키워드  
    "파이썬 프로그래밍",  # 위키피디아의 검색에 적절하지 않은 키워드
]

for query in test_queries:
    print(f"\n=== 검색어: {query} ===")
    try:
        result = wiki_tool.run(query)  # 도구 실행
        print(f"검색 결과: {result}")
        print(f"결과 길이: {len(result)} 문자")
    except Exception as e:
        print(f"오류 발생: {str(e)}")


=== 검색어: BTS ===
검색 결과: Page: BTS
Summary: BTS (Korean: 방탄소년단; RR: Bangtan Sonyeondan; lit. Bulletproof Boy Scouts), also known as the Bangtan Boys, is a South Korean boy band formed in 2010. The band consists of Jin, Suga, J-Hope, RM, Jimin, V, and Jungkook, who co-write or co-produce much of their material. Originally a hip hop group, they expanded their musical style to incorporate a wide range of genres, while their lyrics have focused on subjects including mental health, the troubles of school-age youth and coming of age, loss, the journey towards self-love, individualism, and the consequences of fame and recognition. Their discography and adjacent work has also referenced literature, philosophy and psychology, and includes an alternate universe storyline.
BTS debuted in 2013 under Big Hit Entertainment with the single album 2 Cool 4 Skool. BTS released their first Korean and Japanese-language studio albums, Dark & Wild and Wake Up respectively, in 2014. The group's second Korean

In [14]:
# 계산기 도구 테스트
calc_tool = Tool(
    name="Calculator",
    func=lambda x: eval(x),
    description="수학적 계산을 수행하는 도구. 계산할 수 있는 수식만 입력하세요."
)

# 도구 테스트
result = calc_tool.run("123 * 45")
print("계산 결과:", result)

계산 결과: 5535


### 3. 프롬프트 템플릿 테스트

PromptTemplate: LLM에 전달할 프롬프트의 구조를 정의하는 클래스
- 변수를 포함한 템플릿을 만들어 재사용 가능한 프롬프트 생성

PromptTemplate의 주요 특징:

1. 구성 요소:
   - input_variables: 템플릿에서 사용할 변수들
   - template: 변수를 포함한 프롬프트 구조

2. 장점:
   - 재사용성: 같은 구조의 프롬프트를 다른 입력으로 생성
   - 일관성: 프롬프트 형식의 통일성 유지
   - 유지보수: 프롬프트 구조 변경이 용이

3. 주의사항:
   - 변수명과 실제 전달하는 키값 일치 필요
   - 적절한 컨텍스트 제공
   - 명확한 지시사항 포함

In [15]:
# 1. 기본 프롬프트 템플릿 생성
simple_prompt = PromptTemplate(
    input_variables=["query"],    # 템플릿에서 사용할 변수들의 리스트
                                 # {query}라는 플레이스홀더가 실제 입력으로 대체됨
    
    template="다음 질문에 답해주세요: {query}"  # 프롬프트의 기본 구조
                                               # {변수명}으로 변수 위치 지정
)

# 2. 템플릿 사용 예시
formatted_prompt = simple_prompt.format(
    query="오늘 날씨는 어떤가요?"  # {query}를 실제 질문으로 대체
)
print("포맷된 프롬프트:\n", formatted_prompt)

포맷된 프롬프트:
 다음 질문에 답해주세요: 오늘 날씨는 어떤가요?


In [16]:
#여러 변수를 사용하는 프롬프트
complex_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
주어진 내용: {context}

질문: {question}

위 내용을 바탕으로 답변해주세요:"""
)

# 사용 예시
formatted_complex = complex_prompt.format(
    context="파이썬은 1991년에 만들어진 프로그래밍 언어입니다.",
    question="파이썬은 언제 만들어졌나요?"
)
print("\n복잡한 프롬프트 예시:\n", formatted_complex)


복잡한 프롬프트 예시:
 
주어진 내용: 파이썬은 1991년에 만들어진 프로그래밍 언어입니다.

질문: 파이썬은 언제 만들어졌나요?

위 내용을 바탕으로 답변해주세요:


In [17]:
# 시스템 메시지를 포함한 프롬프트
system_prompt = PromptTemplate(
    input_variables=["instruction", "input"],
    template="""
시스템: 당신은 도움이 되는 AI 어시스턴트입니다.
지시사항: {instruction}

입력: {input}

답변:"""
)

# 사용 예시
formatted_system = system_prompt.format(
    instruction="친절하고 자세하게 설명해주세요.",
    input="인공지능이란 무엇인가요?"
)
print("\n시스템 프롬프트 예시:\n", formatted_system)


시스템 프롬프트 예시:
 
시스템: 당신은 도움이 되는 AI 어시스턴트입니다.
지시사항: 친절하고 자세하게 설명해주세요.

입력: 인공지능이란 무엇인가요?

답변:


In [18]:
# 도구 사용을 위한 프롬프트 템플릿
tool_prompt = PromptTemplate(
    input_variables=["input"],
    template="""다음 도구들을 사용할 수 있습니다:
Calculator: 수학 계산용
Wikipedia: 정보 검색용

작업: {input}

어떤 도구를 사용하시겠습니까?"""
)

# 템플릿 테스트
formatted_tool_prompt = tool_prompt.format(input="2 + 2는 얼마인가요?")
print("도구 프롬프트:", formatted_tool_prompt)

도구 프롬프트: 다음 도구들을 사용할 수 있습니다:
Calculator: 수학 계산용
Wikipedia: 정보 검색용

작업: 2 + 2는 얼마인가요?

어떤 도구를 사용하시겠습니까?


### 4. LLM과 Chain 테스트

In [19]:
# 기본 LLM 테스트
llm = OpenAI(temperature=0, max_tokens=1000)
result = llm.predict("파이썬이란 무엇인가요?")
print("LLM 응답:", result)

# LLM Chain 테스트
chain = LLMChain(
    llm=llm,
    prompt=simple_prompt
)

# Chain 실행
result = chain.run("파이썬의 장점은 무엇인가요?")
print("Chain 결과:", result)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8492\1629227892.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0, max_tokens=1000)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8492\1629227892.py:3: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm.predict("파이썬이란 무엇인가요?")


LLM 응답: 

파이썬은 1991년에 발표된 고급 프로그래밍 언어로, 깔끔하고 간결한 문법을 가지고 있어 쉽게 배우고 사용할 수 있습니다. 인터프리터 언어로서, 컴파일 과정 없이 바로 실행할 수 있어서 빠른 개발이 가능하며, 다양한 운영체제에서 사용할 수 있습니다. 또한 다양한 분야에서 활용되고 있어서 인기 있는 언어 중 하나입니다.


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8492\1629227892.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8492\1629227892.py:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run("파이썬의 장점은 무엇인가요?")


Chain 결과: 

1. 쉬운 학습 곡선: 파이썬은 문법이 간결하고 직관적이어서 쉽게 배울 수 있습니다. 또한 다양한 학습 자료와 커뮤니티가 있어서 학습에 도움이 됩니다.

2. 다양한 용도로 사용 가능: 파이썬은 웹 개발, 데이터 분석, 인공지능, 게임 개발 등 다양한 분야에서 사용될 수 있습니다. 또한 다른 언어와의 통합이 쉬워서 다양한 프로젝트에 유연하게 적용할 수 있습니다.

3. 높은 생산성: 파이썬은 간결한 문법과 다양한 라이브러리를 제공하기 때문에 개발 시간이 짧아지고 생산성이 높아집니다. 또한 디버깅이 쉽고 유지보수가 용이하여 개발자의 부담을 줄여줍니다.

4. 커뮤니티의 활발한 지원: 파이썬은 전 세계적으로 많은 개발자들이 사용하고 있고, 이들이 만든 다양한 라이브러리와 오픈 소스 프로젝트가 있어서 개발에 필요한 자료를 쉽게 얻을 수 있습니다.

5. 크로스 플랫폼 지원: 파이썬은 윈도우, 맥, 리눅스 등 다양한 운영체제에서 동작하며, 이식성이 뛰어나기 때문에 다양한 환경에서 사용할 수 있습니다.

6. 동적 타이핑: 파이썬은 변수의 자료형을 미리 선언하지 않아도 되기 때문에 개발 속도가 빨라집니다. 또한 유연한 자료형 처리가 가능하여 복잡한 작업을 간단하게 처리할 수 있습니다.

7. 높은 확장성: 파이썬은 C나 Java와 같은 언어와의 통합이 쉽고, 다른 언어로 작성된 코드를 파이썬에서 사용할 수 있기 때문에 기존의 코드를 재사용하기에 용이합니다. 또한 다양한 라이브러리를 사용하여 기능을 확장할 수 있습니다.


### 5. 출력 파서 테스트

In [20]:
import re
from typing import Union

class SimpleOutputParser(AgentOutputParser):
    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        # Final Answer 확인
        if "Final Answer:" in text:
            return AgentFinish(
                return_values={"output": text.split("Final Answer:")[-1].strip()},
                log=text
            )
        
        # Action 확인
        match = re.search(r"Action: (.*?)[\n]*Action Input: (.*)", text, re.DOTALL)
        if match:
            action = match.group(1).strip()
            action_input = match.group(2).strip()
            return AgentAction(tool=action, tool_input=action_input, log=text)
        
        # 기본값
        return AgentFinish(
            return_values={"output": text.strip()},
            log=text
        )

# 파서 테스트
parser = SimpleOutputParser()

# 다양한 입력 테스트
# 다양한 입력 테스트 케이스
test_outputs = [
    # 1. 최종 답변 케이스
    """Final Answer: 답은 42입니다.""",

    # 2. 도구 사용 케이스
    """
    Action: Calculator 
    Action Input: 2 + 2
    """,

    # 3. Observation이 포함된 케이스
    """
    Action: Calculator Action Input: 3 * 4
    Observation: 12
    Final Answer: 계산 결과는 12입니다.
    """,

    # 5. 일반 텍스트 응답
    """이것은 일반적인 텍스트 응답입니다.""",

    # 6. 오류가 포함된 Observation 케이스
    """
    Action: Calculator
    Action Input: 2 + x
    Observation: Error: invalid syntax
    Final Answer: 올바른 수식을 입력해주세요.
    """
]

# 파서로 각 케이스 테스트
for i, output in enumerate(test_outputs, 1):
    print(f"\n=== 테스트 케이스 {i} ===")
    print("입력:")
    print(output)
    print("\n파싱 결과:")
    result = parser.parse(output)
    
    # 결과 타입에 따른 상세 정보 출력
    if isinstance(result, AgentAction):
        print(f"결과 유형: AgentAction")
        print(f"선택된 도구: {result.tool}")
        print(f"도구 입력값: {result.tool_input}")
    else:  # AgentFinish
        print(f"결과 유형: AgentFinish")
        print(f"최종 답변: {result.return_values['output']}")
    
    print("-"*50)


=== 테스트 케이스 1 ===
입력:
Final Answer: 답은 42입니다.

파싱 결과:
결과 유형: AgentFinish
최종 답변: 답은 42입니다.
--------------------------------------------------

=== 테스트 케이스 2 ===
입력:

    Action: Calculator 
    Action Input: 2 + 2
    

파싱 결과:
결과 유형: AgentAction
선택된 도구: Calculator
도구 입력값: 2 + 2
--------------------------------------------------

=== 테스트 케이스 3 ===
입력:

    Action: Calculator Action Input: 3 * 4
    Observation: 12
    Final Answer: 계산 결과는 12입니다.
    

파싱 결과:
결과 유형: AgentFinish
최종 답변: 계산 결과는 12입니다.
--------------------------------------------------

=== 테스트 케이스 4 ===
입력:
이것은 일반적인 텍스트 응답입니다.

파싱 결과:
결과 유형: AgentFinish
최종 답변: 이것은 일반적인 텍스트 응답입니다.
--------------------------------------------------

=== 테스트 케이스 5 ===
입력:

    Action: Calculator
    Action Input: 2 + x
    Observation: Error: invalid syntax
    Final Answer: 올바른 수식을 입력해주세요.
    

파싱 결과:
결과 유형: AgentFinish
최종 답변: 올바른 수식을 입력해주세요.
--------------------------------------------------


### 6. 간단한 Agent 실행 테스트

In [21]:
# 간단한 Agent 설정
tools = [wiki_tool, calc_tool]

agent = LLMSingleActionAgent(
    llm_chain=LLMChain(llm=llm, prompt=tool_prompt),
    output_parser=SimpleOutputParser(),
    stop=["\nObservation:"],
    allowed_tools=[tool.name for tool in tools]
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True
)

# Agent 테스트
test_questions = [
    "2 * 3 + 4*999999- 623는 얼마인가요?",
    "파이썬이란 무엇인가요?"
]

for question in test_questions:
    print("\n" + "="*50)
    print("질문:", question)
    try:
        result = agent_executor.run({"input": question})
        print("답변:", result)
    except Exception as e:
        print("오류 발생:", str(e))

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8492\2152164025.py:4: LangChainDeprecationWarning: Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.
  agent = LLMSingleActionAgent(



질문: 2 * 3 + 4*999999- 623는 얼마인가요?


> Entering new AgentExecutor chain...
 Calculator를 사용하겠습니다.

2 * 3 + 4*999999- 623 = 3999938

> Finished chain.
답변: Calculator를 사용하겠습니다.

2 * 3 + 4*999999- 623 = 3999938

질문: 파이썬이란 무엇인가요?


> Entering new AgentExecutor chain...


저는 Wikipedia를 사용하여 파이썬에 대해 알아보겠습니다.

파이썬은 1991년에 발표된 고급 프로그래밍 언어로, 귀도 반 로섬(Guido van Rossum)이 개발하였습니다. 파이썬은 간결하고 읽기 쉬운 문법을 가지고 있어 많은 사람들이 배우기 쉽고 사용하기 쉬운 언어로 인기가 있습니다.

파이썬은 다양한 운영체제에서 사용할 수 있으며, 다양한 분야에서 활용되고 있습니다. 데이터 분석, 인공지능, 웹 개발 등 다양한 분야에서 사용되며, 많은 기업들이 파이썬을 사용하여 소프트웨어를 개발하고 있습니다.

또한 파이썬은 다양한 라이브러리와 프레임워크를 제공하여 개발자들이 빠르고 효율적으로 소프트웨어를 개발할 수 있도록 도와줍니다. 파이썬의 인기는 계속해서 증가하고 있으며, 배우기 쉽고 다양한 분야에서 활용할 수 있는 언어로서 많은 사람들에게 추천되고 있습니다.

> Finished chain.
답변: 저는 Wikipedia를 사용하여 파이썬에 대해 알아보겠습니다.

파이썬은 1991년에 발표된 고급 프로그래밍 언어로, 귀도 반 로섬(Guido van Rossum)이 개발하였습니다. 파이썬은 간결하고 읽기 쉬운 문법을 가지고 있어 많은 사람들이 배우기 쉽고 사용하기 쉬운 언어로 인기가 있습니다.

파이썬은 다양한 운영체제에서 사용할 수 있으며, 다양한 분야에서 활용되고 있습니다. 데이터 분석, 인공지능, 웹 개발 등 다양한 분야에서 사용되며, 많은 기업들이 파이썬을 사용하여 소프트웨어를 개

### 7. 완성된 전체 코드

In [ ]:
# LangChain의 필수 컴포넌트들을 임포트
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser  
# Tool: 에이전트가 사용할 도구를 정의하는 클래스
# AgentExecutor: 에이전트의 실행을 관리하는 클래스
# LLMSingleActionAgent: 한 번에 하나의 행동을 수행하는 에이전트 클래스
# AgentOutputParser: 에이전트 출력을 파싱하는 기본 클래스

from langchain.prompts import StringPromptTemplate, PromptTemplate  
# 프롬프트 템플릿 관련 클래스들
# StringPromptTemplate: 문자열 기반 프롬프트 템플릿의 기본 클래스
# PromptTemplate: 변수를 포함한 프롬프트 템플릿을 만드는 클래스

from langchain.chains import LLMChain  # LLM과 프롬프트를 연결하는 체인 클래스
from langchain.llms import OpenAI  # OpenAI의 LLM을 사용하기 위한 클래스
from langchain.schema import AgentAction, AgentFinish  # 에이전트의 행동과 종료를 나타내는 클래스
from typing import List, Union  # 파이썬 타입 힌팅을 위한 클래스
import re  # 정규 표현식 처리를 위한 모듈
from langchain.utilities import WikipediaAPIWrapper  # Wikipedia API 래퍼 클래스

def get_wikipedia_tool():
    """Wikipedia 검색 도구를 생성하는 함수
    Wikipedia API를 통해 정보를 검색하고 반환하는 도구를 생성
    
    Returns:
        Tool: 설정된 Wikipedia 검색 도구
    """
    wikipedia = WikipediaAPIWrapper()  # Wikipedia API 인스턴스 생성
    return Tool(
        name="Wikipedia",  # 도구의 식별자
        func=wikipedia.run,  # Wikipedia 검색을 실행할 함수
        description="Wikipedia에서 정보를 검색할 때 유용한 도구. 검색할 수 있는 주제나 키워드를 입력하세요."
    )

def get_calculator_tool():
    """계산기 도구를 생성하는 함수
    수학적 표현식을 평가하여 결과를 반환하는 도구를 생성
    
    Returns:
        Tool: 설정된 계산기 도구
    """
    return Tool(
        name="Calculator",
        func=lambda x: eval(x),  # 문자열로 된 수학 표현식을 평가
        description="수학적 계산을 수행하는 도구. 계산할 수 있는 수식만 입력하세요."
    )

class CustomOutputParser(AgentOutputParser):
    """LLM 출력을 파싱하는 커스텀 파서 클래스
    
    LLM의 출력을 분석하여 다음 행동(AgentAction) 또는 
    최종 답변(AgentFinish)으로 변환하는 파서
    """
    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        """LLM 출력 텍스트를 파싱하는 메소드
        
        Args:
            text (str): LLM이 생성한 출력 텍스트
        
        Returns:
            Union[AgentAction, AgentFinish]: 다음 행동 또는 최종 답변
        """
        # 최종 답변이 포함된 경우 처리
        if "Final Answer:" in text:
            return AgentFinish(
                return_values={"output": text.split("Final Answer:")[-1].strip()},
                log=text
            )
        
        # Action과 Input이 포함된 경우 처리
        match = re.search(r"Action: (.*?)[\n]*Action Input: (.*)", text, re.DOTALL)
        if not match:
            return AgentFinish(
                return_values={"output": text},
                log=text
            )
        
        # 매칭된 결과를 AgentAction으로 반환
        action = match.group(1).strip()
        action_input = match.group(2).strip()
        return AgentAction(tool=action, tool_input=action_input, log=text)

def setup_agent():
    """Agent를 설정하고 반환하는 함수
    
    모든 구성 요소(도구, 프롬프트, LLM, 파서)를 조합하여
    완전한 Agent를 생성하고 반환
    
    Returns:
        AgentExecutor: 설정된 Agent 실행기
    """
    # 사용할 도구들 초기화
    tools = [get_wikipedia_tool(), get_calculator_tool()]
    
    # 프롬프트 템플릿 정의
    prompt = PromptTemplate(
        input_variables=["input"],  # 템플릿에서 사용할 변수
        template="""다음 작업을 수행하기 위해 주어진 도구들을 사용하세요:

사용 가능한 도구들:
Calculator: 수학적 계산을 수행하는 도구입니다. 예: 123*45, 67+89, 100-50 등의 수식을 계산할 수 있습니다.
Wikipedia: Wikipedia에서 정보를 검색할 때 유용한 도구입니다. 주제나 키워드를 입력하세요.

작업: {input}

다음 규칙을 따르세요:
1. 수학 계산이 필요한 경우 반드시 Calculator 도구를 사용하세요.
2. 정보 검색이 필요한 경우 Wikipedia 도구를 사용하세요.

답변 형식:
1. 도구를 사용할 경우:
   Action: [Calculator 또는 Wikipedia]
   Action Input: [도구에 전달할 입력]

2. 최종 답변의 경우:
   Final Answer: [답변 내용]

답변:"""
    )
    
    # LLM 체인 생성
    llm_chain = LLMChain(
        llm=OpenAI(temperature=0),  # temperature=0: 가장 결정적인 출력 생성
        prompt=prompt  # 위에서 정의한 프롬프트 사용
    )
    
    # Agent 생성
    agent = LLMSingleActionAgent(
        llm_chain=llm_chain,  # LLM과 프롬프트를 연결한 체인
        output_parser=CustomOutputParser(),  # 출력 파싱을 위한 파서
        stop=["\nObservation:"],  # 생성 중단 토큰
        allowed_tools=[tool.name for tool in tools]  # 사용 가능한 도구들
    )
    
    # Agent 실행기 생성 및 반환
    return AgentExecutor.from_agent_and_tools(
        agent=agent,
        tools=tools,
        verbose=True  # 실행 과정을 상세히 출력
    )

# 메인 실행 부분
if __name__ == "__main__":
    # Agent 설정
    agent_executor = setup_agent()
    
    # 테스트할 질문들
    questions = [
        "BTS의 히트곡은?",  # Wikipedia 검색 테스트
        "인공지능의 역사는 어떻게 되나요?",  # Wikipedia 검색 테스트
        "127*4 - 99는 얼마인가요?",  # 계산기 테스트
    ]
    
    # 각 질문에 대해 Agent 실행
    for question in questions:
        print("\n" + "="*50)
        print(f"질문: {question}")
        result = agent_executor.run({"input": question})  # Agent 실행
        print(f"답변: {result}")
        print("="*50)